In [1]:
# default_exp core.scraping.event

# Event

> Scrapes CZ event information.

In [2]:
#hide
from nbdev.showdoc import *

In [3]:
#export

from czapi.core.scraping.base import make_soup
from czapi.core.scraping.constants import LINESCORE_SOUP_TYPE, BOXSCORE_SOUP_TYPE
from bs4 import BeautifulSoup
from typing import Union, Optional

In [4]:
# hide

url = 'https://www.curlingzone.com/event.php?view=Main&eventid=6938'
soup = make_soup(url)

In [5]:
# exporti

def _get_event_name_from_linescore_page(
     soup : BeautifulSoup
)->str:
    """Returns the name of the event based on the event_id. Built so for larger-scale scraping less API calls are made."""
    return soup.find('title').string

def _get_event_name_from_boxscore_page(
    
    soup : BeautifulSoup

)->str:
    """Returns the name of the vent based on the game_id. Build so for larger-scale scrpaing less API calls are made."""
    
    return soup.find('h3',attrs={'class':'entry-title-widget'}).string

In [6]:
# exporti


def _get_event_url(
     cz_event_id : Union[str,int]
    ,cz_draw_id : Optional[Union[str,int]] = None
)->str:
    """Returns the cz event page url."""
    
    if cz_draw_id is None:
        return 'https://curlingzone.com/event.php?view=Scores&eventid=%s#1'%cz_event_id
    
    return 'https://curlingzone.com/event.php?eventid=%s&view=Scores&showdrawid=%s#1'%(cz_event_id,cz_draw_id)

def _get_game_url(
    cz_game_id : Union[str,int]
)->str:
    """Returns the cz game page url."""

    return 'https://www.curlingzone.com/game.php?1=1&showgameid=%s#1'%cz_game_id

def get_url(

     soup_type : str
    ,**url_kwargs

)->str:
    """Helper function for returning the correct CurlingZone URL."""
    
    soup_type = soup_type.lower()
    
    if soup_type == LINESCORE_SOUP_TYPE:
        return _get_event_url(**url_kwargs)
    elif soup_type == BOXSCORE_SOUP_TYPE:
        return _get_game_url(**url_kwargs)
    else:
        raise NotImplementedError("%s soup type is not implemented."%soup_type)
        
def _get_event_name(
    
     soup : BeautifulSoup
    ,soup_type : str


)->str:
    soup_type = soup_type.lower()
    
    if soup_type == LINESCORE_SOUP_TYPE:
        return _get_event_name_from_linescore_page(soup=soup)
    elif soup_type == BOXSCORE_SOUP_TYPE:
        return _get_event_name_from_boxscore_page(soup=soup)
    else:
        raise NotImplementedError("%s soup type is not implemented."%soup_type)

In [7]:
# export
def get_event_name(
     cz_event_id : Optional[Union[str,int]] = None
    ,cz_draw_id : Optional[Union[str,int]] = None
    ,cz_game_id : Optional[Union[str,int]] = None
    ,**request_kwargs
)->str:
    """Returns the name of the event based on the cz_event_id or the cz_game_id."""
    
    # TODO : how to handle situations where an event_id is valid but that event hasn't been conducted yet?
    # e.g. event_id = 12312321312 returns 'CurlingZone – Everything Curling'
    
    ids = [cz_event_id,cz_game_id]
    if all(ids) or not any(ids):
        raise ValueError("One of cz_event_id or cz_game_id can be a non NoneType.")
    
    if cz_event_id:
        soup_type = LINESCORE_SOUP_TYPE
        url_kwargs = {
             'cz_event_id' : cz_event_id
            ,'cz_draw_id' : cz_draw_id
        }
        
    
    else:
        soup_type = BOXSCORE_SOUP_TYPE
        url_kwargs = {
             'cz_game_id' : cz_game_id
        }
        url = _get_game_url(cz_game_id = cz_game_id)

    url = get_url(soup_type = soup_type,**url_kwargs)
    soup = make_soup(url=url,**request_kwargs)
    
    return _get_event_name(soup=soup,soup_type = soup_type)

In [8]:
#hide
soup = make_soup('https://curlingzone.com/event.php?view=Main&eventid=6900#1')
soup.find('title').string

'Scotties Tournament of Hearts'

In [9]:
# tests

# scotties 
assert get_event_name(cz_event_id = '6900') == 'Scotties Tournament of Hearts'
assert get_event_name(cz_game_id = 297288) == 'Scotties Tournament of Hearts'

# random event
assert get_event_name(cz_event_id = '6938',cz_draw_id = 1) == 'Hokkaido Bank Curling Classic'
assert get_event_name(cz_game_id = 284807) == 'Hokkaido Bank Curling Classic'

try:
    get_event_name(1,2)
except ValueError as e:
    assert isinstance(e,ValueError)
    
try:
    get_event_name()
except ValueError as e:
    assert isinstance(e,ValueError)

In [10]:
# exporti

def _get_event_date_from_linescore_page(

     soup : BeautifulSoup
)->str:
    """Returns the dates of the event from the passed soup. Built so for larger-scale scraping less API calls are made."""
    return soup.find(name='div',attrs={'class':'badge-widget'}).string

def _get_event_date_from_boxscore_page(

    soup : BeautifulSoup

)->str:
    """Returns the dates of the event from the passed soup. Built so for larger-scale scraping less API calls are made."""
    return soup.find('div',attrs={'class':'badge-widget'}).string

In [11]:
# exporti

def _get_event_date(
    
     soup : BeautifulSoup
    ,soup_type : str


)->str:
    soup_type = soup_type.lower()
    
    if soup_type == LINESCORE_SOUP_TYPE:
        return _get_event_date_from_linescore_page(soup=soup)
    elif soup_type == BOXSCORE_SOUP_TYPE:
        return _get_event_date_from_boxscore_page(soup=soup)
    else:
        raise NotImplementedError("%s soup type is not implemented."%soup_type)

In [12]:
# export

def get_event_date(

     cz_event_id : Optional[Union[str,int]] = None
    ,cz_draw_id : Optional[Union[str,int]] = None
    ,cz_game_id : Optional[Union[str,int]] = None
    ,**request_kwargs

)->str:
    """Returns the dates of the event based on the cz_event_id or the cz_game_id."""
    
    # TODO : how to handle situations where an event_id is valid but that event hasn't been conducted yet?
    # e.g. event_id = 12312321312 returns 'Recent'
    
    ids = [cz_event_id,cz_game_id]
    if all(ids) or not any(ids):
        raise ValueError("One of cz_event_id or cz_game_id can be a non NoneType.")
    
    if cz_event_id:
        soup_type = LINESCORE_SOUP_TYPE
        url_kwargs = {
             'cz_event_id' : cz_event_id
            ,'cz_draw_id' : cz_draw_id
        }

        

    else:
        soup_type = BOXSCORE_SOUP_TYPE
        url_kwargs = {
            'cz_game_id' : cz_game_id
        }

        
    url =  get_url(soup_type = soup_type,**url_kwargs)
    soup = make_soup(url=url,**request_kwargs)
    
    return _get_event_date(soup=soup,soup_type=soup_type)

In [13]:
# tests

# scotties
assert get_event_date(cz_event_id = '6900',cz_draw_id = 1) == 'Jan 28 - Feb 6, 2022'
assert get_event_date(cz_game_id = 297288) == 'Jan 28 - Feb 6, 2022'

try:
    get_event_date()
except ValueError as e:
    assert isinstance(e,ValueError)

try:
    get_event_date(1,2)
except ValueError as e:
    assert isinstance(e,ValueError)
    
# random event
assert get_event_date(cz_event_id = '6938') == 'Aug 19 - 22, 2021'
assert get_event_date(cz_game_id = 284807) == 'Aug 19 - 22, 2021'